<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 9.7: Text Classification
INSTRUCTIONS:
- Run the cells
- Observe and understand the results
- Answer the questions

# Import libraries

In [3]:
## Import Libraries
import numpy as np
import pandas as pd

import string
import spacy

from collections import Counter

from sklearn.decomposition import LatentDirichletAllocation
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC

# import warnings
# warnings.filterwarnings('ignore')

'''
conda install -c conda-forge spacy
'''

'\nconda install -c conda-forge spacy\n'

# Load data

Sample:

    __label__2 Stuning even for the non-gamer: This sound ...
    __label__2 The best soundtrack ever to anything.: I'm ...
    __label__2 Amazing!: This soundtrack is my favorite m ...
    __label__2 Excellent Soundtrack: I truly like this so ...
    __label__2 Remember, Pull Your Jaw Off The Floor Afte ...
    __label__2 an absolute masterpiece: I am quite sure a ...
    __label__1 Buyer beware: This is a self-published boo ...
    . . .
    
There are only two **labels**:
- `__label__1`
- `__label__2`

In [8]:
## Loading the data
trainDF = pd.read_fwf(
    filepath_or_buffer = '../Data/corpus.txt',
    colspecs = [(9, 10),   # label: get only the numbers 1 or 2
                (11, 9000) # text: makes the it big enought to get to the end of the line
               ], 
    header = 0,
    names = ['label', 'text'],
    lineterminator = '\n'
)

# convert label from [1, 2] to [0, 1]
trainDF['label'] = trainDF['label'] - 1

## Inspect the data

In [11]:
# ANSWER
print(trainDF.info())
print(trainDF.sample(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9999 entries, 0 to 9998
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   9999 non-null   int64 
 1   text    9999 non-null   object
dtypes: int64(1), object(1)
memory usage: 156.4+ KB
None
      label                                               text
4668      1  Not for those looking for an uplifting read.: ...
5568      1  The cliches were invented right here: There ar...
5449      0  Avoid at all cost !!: Some movies are bad and ...
2935      1  Dark of the Moon DVD: Great value for money. I...
6173      1  Life Drawing: Techniques in Action: I teach Fi...
7073      1  A Majority of One: This movie is just wonderfu...
6192      1  Not the same as pitchure: The book dose the jo...
8730      0  i hated this movie: I had the misfortune of pa...
2112      1  Great: My son just started getting into superh...
8558      1  My first Pratchett book -- I'm hooked!: A well...


## Split the data into train and test

In [17]:
## ANSWER
## split the dataset
X = trainDF['text']
y = trainDF['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature Engineering

## Count Vectors as features

In [27]:
from sklearn.feature_extraction.text import CountVectorizer

# create a count vectorizer object
count_vect = CountVectorizer(token_pattern = r'\w{1,}')
# Learn a vocabulary dictionary of all tokens in the raw documents
count_vect.fit(trainDF['text'])
# Transform documents to document-term matrix.
X_train_count = count_vect.transform(X_train)
X_test_count = count_vect.transform(X_test)

In [30]:
X_train_count

<7999x31661 sparse matrix of type '<class 'numpy.int64'>'
	with 466218 stored elements in Compressed Sparse Row format>

## TF-IDF Vectors as features
- Word level
- N-Gram level
- Character level

### Word level

In [33]:
%%time
from sklearn.feature_extraction.text import TfidfVectorizer
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer = 'word',
                             token_pattern = r'\w{1,}',
                             max_features = 5000)
print(tfidf_vect)

tfidf_vect.fit(trainDF['text'])
X_train_tfidf = tfidf_vect.transform(X_train)
X_test_tfidf  = tfidf_vect.transform(X_test)

TfidfVectorizer(max_features=5000, token_pattern='\\w{1,}')
Wall time: 835 ms


In [31]:
X_train_tfidf

<7999x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 421773 stored elements in Compressed Sparse Row format>

### N-Gram level

In [34]:
%%time
# ngram level tf-idf
tfidf_vect_ngram = TfidfVectorizer(analyzer = 'word',
                                   token_pattern = r'\w{1,}',
                                   ngram_range = (2, 3),
                                   max_features = 5000)
print(tfidf_vect_ngram)

tfidf_vect_ngram.fit(trainDF['text'])
X_train_tfidf_ngram = tfidf_vect_ngram.transform(X_train)
X_test_tfidf_ngram  = tfidf_vect_ngram.transform(X_test)

TfidfVectorizer(max_features=5000, ngram_range=(2, 3), token_pattern='\\w{1,}')
Wall time: 4.43 s


In [35]:
X_train_tfidf_ngram

<7999x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 278377 stored elements in Compressed Sparse Row format>

### Characters level

In [36]:
%%time
# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer = 'char',
                                         token_pattern = r'\w{1,}',
                                         ngram_range = (2, 3),
                                         max_features = 5000)
print(tfidf_vect_ngram_chars)

tfidf_vect_ngram_chars.fit(trainDF['text'])
X_train_tfidf_ngram_chars = tfidf_vect_ngram_chars.transform(X_train)
X_test_tfidf_ngram_chars  = tfidf_vect_ngram_chars.transform(X_test)

TfidfVectorizer(analyzer='char', max_features=5000, ngram_range=(2, 3),
                token_pattern='\\w{1,}')


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:506: UserWarning: The parameter 'token_pattern' will not be used since 'analyzer' != 'word'
  warnings.warn("The parameter 'token_pattern' will not be used"


Wall time: 6.27 s


In [37]:
X_train_tfidf_ngram_chars

<7999x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 3604820 stored elements in Compressed Sparse Row format>

## Text / NLP based features

Create some other features:

- Char_Count = Number of Characters in Text
- Word Count = Number of Words in Text
- Word Density = Average Number of Char in Words
- Punctuation Count = Number of Punctuation in Text
- Title Word Count = Number of Words in Title
- Uppercase Word Count = Number of Upperwords in Text

In [45]:
%%time
# ANSWER
trainDF['char_count']  = trainDF['text'].apply(len)
trainDF['word_count']  = trainDF['text'].apply(lambda x: len(x.split()))
trainDF['word_density']= trainDF['char_count']/(trainDF['word_count']+1)
trainDF['punc_count']  = trainDF['text'].apply(lambda x: len(''.join(_ for _ in x if _ in string.punctuation)))
trainDF['title_count'] = trainDF['text'].apply(lambda x: len([w for w in x.split() if w.istitle()]))
trainDF['upper_count'] = trainDF['text'].apply(lambda x: len([w for w in x.split() if w.isupper()]))

Wall time: 406 ms


In [46]:
trainDF.head()

,label,text,char_count,word_count,word_density,punc_count,title_count,upper_count
0,1,The best soundtrack ever to anything.: I'm rea...,509,97,5.193878,14,7,3
1,1,Amazing!: This soundtrack is my favorite music...,760,129,5.846154,40,24,4
2,1,Excellent Soundtrack: I truly like this soundt...,743,118,6.243697,33,52,4
3,1,"Remember, Pull Your Jaw Off The Floor After He...",481,87,5.465909,22,30,0
4,1,an absolute masterpiece: I am quite sure any o...,825,142,5.769231,35,14,3


In [52]:
trainDF.sample(5)

,label,text,char_count,word_count,word_density,punc_count,title_count,upper_count
4321,0,Too Small: Hard to tell from the picture what ...,201,40,4.902439,6,6,1
6619,0,"Worst ever: I would have rated this minus 5, b...",381,77,4.884615,14,8,5
8262,1,asvab for dummies: great book..i have learned ...,243,45,5.282609,6,0,2
9357,1,Mysterious hospital deaths: Judith McMonigle F...,882,149,5.880000,15,17,0
7313,0,Fahrenheit 451: We thought this book was a goo...,594,120,4.909091,17,10,2


In [54]:
## load spaCy
import spacy
'''# python -m spacy download en_core_web_sm'''
nlp = spacy.load('en_core_web_sm')     

Part of Speech in **SpaCy**

    POS   DESCRIPTION               EXAMPLES
    ----- ------------------------- ---------------------------------------------
    ADJ   adjective                 big, old, green, incomprehensible, first
    ADP   adposition                in, to, during
    ADV   adverb                    very, tomorrow, down, where, there
    AUX   auxiliary                 is, has (done), will (do), should (do)
    CONJ  conjunction               and, or, but
    CCONJ coordinating conjunction  and, or, but
    DET   determiner                a, an, the
    INTJ  interjection              psst, ouch, bravo, hello
    NOUN  noun                      girl, cat, tree, air, beauty
    NUM   numeral                   1, 2017, one, seventy-seven, IV, MMXIV
    PART  particle                  's, not,
    PRON  pronoun                   I, you, he, she, myself, themselves, somebody
    PROPN proper noun               Mary, John, London, NATO, HBO
    PUNCT punctuation               ., (, ), ?
    SCONJ subordinating conjunction if, while, that
    SYM   symbol                    $, %, §, ©, +, −, ×, ÷, =, :), 😝
    VERB  verb                      run, runs, running, eat, ate, eating
    X     other                     sfpksdpsxmsa
    SPACE space
    
Find out number of Adjective, Adverb, Noun, Numeric, Pronoun, Proposition, Verb.

    Hint:
    1. Convert text to spacy document
    2. Use pos_
    3. Use Counter 

In [55]:
# Initialise some columns for feature's counts
trainDF['adj_count'] = 0
trainDF['adv_count'] = 0
trainDF['noun_count'] = 0
trainDF['num_count'] = 0
trainDF['pron_count'] = 0
trainDF['propn_count'] = 0
trainDF['verb_count'] = 0

In [56]:
%%time
# ANSWER
for i in range(trainDF.shape[0]):
    # convert into a spaCy document
    doc = nlp(trainDF.iloc[i]['text'])
    # initialise feature counters
    c = Counter([t.pos_ for t in doc])
    
    trainDF.at[i, 'adj_count'] = c['ADJ']
    trainDF.at[i, 'adv_count'] = c['ADV']
    trainDF.at[i, 'noun_count'] = c['NOUN']
    trainDF.at[i, 'num_count'] = c['NUM']
    trainDF.at[i, 'pron_count'] = c['PRON']
    trainDF.at[i, 'propn_count'] = c['PROPN']
    trainDF.at[i, 'verb_count'] = c['VERB']
    

Wall time: 2min 19s


In [0]:
cols = [
    'char_count', 'word_count', 'word_density',
    'punctuation_count', 'title_word_count',
    'uppercase_word_count', 'adj_count',
    'adv_count', 'noun_count', 'num_count',
    'pron_count', 'propn_count', 'verb_count']

trainDF[cols].sample(5)

,char_count,word_count,word_density,punctuation_count,title_word_count,uppercase_word_count,adj_count,adv_count,noun_count,num_count,pron_count,propn_count,verb_count
4986,240,45,5.217391,9,3,0,3,4,8,1,5,0,14
9212,158,29,5.266667,6,5,1,3,1,4,1,1,3,8
6726,645,122,5.243902,20,18,6,12,13,25,1,8,5,24
2925,182,35,5.055556,8,4,4,8,4,2,0,5,0,8
124,371,66,5.537313,21,10,1,9,4,14,3,6,2,11


## Topic Models as features

In [57]:
%%time
# train a LDA Model
lda_model = LatentDirichletAllocation(n_components = 20, learning_method = 'online', max_iter = 20)

X_topics = lda_model.fit_transform(X_train_count)
topic_word = lda_model.components_ 
vocab = count_vect.get_feature_names()

Wall time: 1min 8s


In [58]:
# view the topic models
n_top_words = 10
topic_summaries = []
print('Group Top Words')
print('-----', '-'*80)
for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
    top_words = ' '.join(topic_words)
    topic_summaries.append(top_words)
    print('  %3d %s' % (i, top_words))

Group Top Words
----- --------------------------------------------------------------------------------
    0 these them boots they pair boot items run shoes are
    1 et le philadelphia les est manon il des birds bless
    2 size yoga tour video beach legs hip chart correctly bowie
    3 card memory camera error magazine stands news chips adopted serve
    4 theory practice lady wayne hospital treasure lesson gibson dell principles
    5 edition printer missing print page mystery copy blu tales catholic
    6 hollywood diane lane pack force strongly rod seller worthy wanting
    7 emma celiac scent exuviance shades guinea papua alcohol prefered tanks
    8 movie effects special ear fit large acting crap jawbone haunting
    9 battery charger charging g4 recipes powerbook 00 ibook emarker wild
   10 la de y en el que harry del un es
   11 manson cave his he japanese bear ayla clan charles van
   12 cap stockings oriented frm wagner hopeful garp landscapes substandard nomad
   13 latest 

# Modelling

In [59]:
## helper function

def train_model(classifier, feature_vector_train, label, feature_vector_valid):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)

    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)

    return accuracy_score(predictions, y_test)

In [60]:
# Keep the results in a dataframe
results = pd.DataFrame(columns = ['Count Vectors',
                                  'WordLevel TF-IDF',
                                  'N-Gram Vectors',
                                  'CharLevel Vectors'])

## Naive Bayes Classifier

In [61]:
%%time
# Naive Bayes on Count Vectors
accuracy1 = train_model(MultinomialNB(), X_train_count, y_train, X_test_count)
print('NB, Count Vectors    : %.4f\n' % accuracy1)

NB, Count Vectors    : 0.8540

Wall time: 8 ms


In [62]:
%%time
# Naive Bayes on Word Level TF IDF Vectors
accuracy2 = train_model(MultinomialNB(), X_train_tfidf, y_train, X_test_tfidf)
print('NB, WordLevel TF-IDF : %.4f\n' % accuracy2)

NB, WordLevel TF-IDF : 0.8600

Wall time: 7.13 ms


In [63]:
%%time
# Naive Bayes on Ngram Level TF IDF Vectors
accuracy3 = train_model(MultinomialNB(), X_train_tfidf_ngram, y_train, X_test_tfidf_ngram)
print('NB, N-Gram Vectors   : %.4f\n' % accuracy3)

NB, N-Gram Vectors   : 0.8400

Wall time: 6 ms


In [64]:
%%time
# # Naive Bayes on Character Level TF IDF Vectors
accuracy4 = train_model(MultinomialNB(), X_train_tfidf_ngram_chars, y_train, X_test_tfidf_ngram_chars)
print('NB, CharLevel Vectors: %.4f\n' % accuracy4)

NB, CharLevel Vectors: 0.8180

Wall time: 21 ms


In [65]:
results.loc['Naïve Bayes'] = {
    'Count Vectors': accuracy1,
    'WordLevel TF-IDF': accuracy2,
    'N-Gram Vectors': accuracy3,
    'CharLevel Vectors': accuracy4}

## Linear Classifier

In [66]:
%%time
# Linear Classifier on Count Vectors
accuracy1 = train_model(LogisticRegression(solver = 'lbfgs', max_iter = 350), X_train_count, y_train, X_test_count)
print('LR, Count Vectors    : %.4f\n' % accuracy1)

LR, Count Vectors    : 0.8520

Wall time: 1.66 s


In [67]:
%%time
# Linear Classifier on Word Level TF IDF Vectors
accuracy2 = train_model(LogisticRegression(solver = 'lbfgs', max_iter = 100), X_train_tfidf, y_train, X_test_tfidf)
print('LR, WordLevel TF-IDF : %.4f\n' % accuracy2)

LR, WordLevel TF-IDF : 0.8730

Wall time: 79.7 ms


In [68]:
%%time
# Linear Classifier on Ngram Level TF IDF Vectors
accuracy3 = train_model(LogisticRegression(solver = 'lbfgs', max_iter = 100), X_train_tfidf_ngram, y_train, X_test_tfidf_ngram)
print('LR, N-Gram Vectors   : %.4f\n' % accuracy3)

LR, N-Gram Vectors   : 0.8360

Wall time: 43.3 ms


In [69]:
%%time
# Linear Classifier on Character Level TF IDF Vectors
accuracy4 = train_model(LogisticRegression(solver = 'lbfgs', max_iter = 100), X_train_tfidf_ngram_chars, y_train, X_test_tfidf_ngram_chars)
print('LR, CharLevel Vectors: %.4f\n' % accuracy4)

LR, CharLevel Vectors: 0.8485

Wall time: 304 ms


In [70]:
results.loc['Logistic Regression'] = {
    'Count Vectors': accuracy1,
    'WordLevel TF-IDF': accuracy2,
    'N-Gram Vectors': accuracy3,
    'CharLevel Vectors': accuracy4}

## Support Vector Machine

In [71]:
%%time
# Support Vector Machine on Count Vectors
accuracy1 = train_model(LinearSVC(), X_train_count, y_train, X_test_count)
print('SVM, Count Vectors    : %.4f\n' % accuracy1)

SVM, Count Vectors    : 0.8345

Wall time: 526 ms


In [72]:
%%time
# Support Vector Machine on Word Level TF IDF Vectors
accuracy2 = train_model(LinearSVC(), X_train_tfidf, y_train, X_test_tfidf)
print('SVM, WordLevel TF-IDF : %.4f\n' % accuracy2)

SVM, WordLevel TF-IDF : 0.8610

Wall time: 42.6 ms


In [73]:
%%time
# Support Vector Machine on Ngram Level TF IDF Vectors
accuracy3 = train_model(LinearSVC(), X_train_tfidf_ngram, y_train, X_test_tfidf_ngram)
print('SVM, N-Gram Vectors   : %.4f\n' % accuracy3)

SVM, N-Gram Vectors   : 0.8210

Wall time: 41.1 ms


In [74]:
%%time
# Support Vector Machine on Character Level TF IDF Vectors
accuracy4 = train_model(LinearSVC(), X_train_tfidf_ngram_chars, y_train, X_test_tfidf_ngram_chars)
print('SVM, CharLevel Vectors: %.4f\n' % accuracy4)

SVM, CharLevel Vectors: 0.8570

Wall time: 452 ms


In [75]:
results.loc['Support Vector Machine'] = {
    'Count Vectors': accuracy1,
    'WordLevel TF-IDF': accuracy2,
    'N-Gram Vectors': accuracy3,
    'CharLevel Vectors': accuracy4}

## Bagging Models

In [76]:
%%time
# Bagging (Random Forest) on Count Vectors
accuracy1 = train_model(RandomForestClassifier(n_estimators = 100), X_train_count, y_train, X_test_count)
print('RF, Count Vectors    : %.4f\n' % accuracy1)

RF, Count Vectors    : 0.8270

Wall time: 10.1 s


In [77]:
%%time
# Bagging (Random Forest) on Word Level TF IDF Vectors
accuracy2 = train_model(RandomForestClassifier(n_estimators = 100), X_train_tfidf, y_train, X_test_tfidf)
print('RF, WordLevel TF-IDF : %.4f\n' % accuracy2)

RF, WordLevel TF-IDF : 0.8205

Wall time: 4.58 s


In [78]:
%%time
# Bagging (Random Forest) on Ngram Level TF IDF Vectors
accuracy3 = train_model(RandomForestClassifier(n_estimators = 100), X_train_tfidf_ngram, y_train, X_test_tfidf_ngram)
print('RF, N-Gram Vectors   : %.4f\n' % accuracy3)

RF, N-Gram Vectors   : 0.7850

Wall time: 4.53 s


In [79]:
%%time
# Bagging (Random Forest) on Character Level TF IDF Vectors
accuracy4 = train_model(RandomForestClassifier(n_estimators = 100), X_train_tfidf_ngram_chars, y_train, X_test_tfidf_ngram_chars)
print('RF, CharLevel Vectors: %.4f\n' % accuracy4)

RF, CharLevel Vectors: 0.7835

Wall time: 18.3 s


In [80]:
results.loc['Random Forest'] = {
    'Count Vectors': accuracy1,
    'WordLevel TF-IDF': accuracy2,
    'N-Gram Vectors': accuracy3,
    'CharLevel Vectors': accuracy4}

## Boosting Models

In [81]:
%%time
# Gradient Boosting on Count Vectors
accuracy1 = train_model(GradientBoostingClassifier(), X_train_count, y_train, X_test_count)
print('GB, Count Vectors    : %.4f\n' % accuracy1)

GB, Count Vectors    : 0.7990

Wall time: 22.2 s


In [82]:
%%time
# Gradient Boosting on Word Level TF IDF Vectors
accuracy2 = train_model(GradientBoostingClassifier(), X_train_tfidf, y_train, X_test_tfidf)
print('GB, WordLevel TF-IDF : %.4f\n' % accuracy2)

GB, WordLevel TF-IDF : 0.7950

Wall time: 8.83 s


In [83]:
%%time
# Gradient Boosting on Ngram Level TF IDF Vectors
accuracy3 = train_model(GradientBoostingClassifier(), X_train_tfidf_ngram, y_train, X_test_tfidf_ngram)
print('GB, N-Gram Vectors   : %.4f\n' % accuracy3)

GB, N-Gram Vectors   : 0.7370

Wall time: 5.58 s


In [84]:
%%time
# Gradient Boosting on Character Level TF IDF Vectors
accuracy4 = train_model(GradientBoostingClassifier(), X_train_tfidf_ngram_chars, y_train, X_test_tfidf_ngram_chars)
print('GB, CharLevel Vectors: %.4f\n' % accuracy4)

GB, CharLevel Vectors: 0.8020

Wall time: 1min 41s


In [85]:
results.loc['Gradient Boosting'] = {
    'Count Vectors': accuracy1,
    'WordLevel TF-IDF': accuracy2,
    'N-Gram Vectors': accuracy3,
    'CharLevel Vectors': accuracy4}

In [86]:
results

,Count Vectors,WordLevel TF-IDF,N-Gram Vectors,CharLevel Vectors
Naïve Bayes,0.8540,0.8600,0.840,0.8180
Logistic Regression,0.8520,0.8730,0.836,0.8485
Support Vector Machine,0.8345,0.8610,0.821,0.8570
Random Forest,0.8270,0.8205,0.785,0.7835
Gradient Boosting,0.7990,0.7950,0.737,0.8020
